# Export the trained model to a Docker image


In this section of the lab we will export the trained model to a Docker image. We will use CVS Python SDK to download the export package and Docker CLI to create the image. Then we will deploy the image to Azure Container Instance. 


## Export the iteration
Exporting a model is a two-step process. First you must request the export. It is an asynchronous call. You have to periodically check the status of the request and when the export package is ready you can download it using the returned URI.


Navigate to **Project Settings** and copy and paste *Project ID* and *Training Key*. Also, cut and paste the name of the iteration which was trained with the compact domain.


In [3]:
#PROJECT_ID = '<your project id>'
#TRAINING_ID = '<your training key>'

PROJECT_ID = '556e602a-e747-4fb9-b694-63ac2a06cc8d'
TRAINING_KEY = 'fc2ca3cbae90475eaa595f8a4dfcda86'
ITERATION_NAME = 'Iteration 4'

In [3]:
PROJECT_ID = '<your project id>'
TRAINING_ID = '<your training key>'
ITERATION_NAME = '<your iteration name>'

### Request the export

In [ ]:
import azure.cognitiveservices.vision.customvision.training as training_api


trainer = training_api.TrainingApi(TRAINING_KEY)

# Search through iterations collection to find the one you want to export
iteration_id = None
for iteration in trainer.get_iterations(PROJECT_ID):
    if iteration.name == ITERATION_NAME:
        iteration_id = iteration.id
        break

# Request the export
if iteration_id == None:
    print("Could not find the iteration")
else:
    print("Requesting export for Iteration ID: {0}".format(iteration_id))
    try:
        trainer.export_iteration(PROJECT_ID, iteration_id, platform='DockerFile', flavor='Linux')
    except:
        print("Error while requesting export. Most likely the export already exists")
    

### Wait till the export is ready


In [ ]:
import time

# Periodically check the status of request
download_uri = None
while True:
    exports = trainer.get_exports(PROJECT_ID, iteration_id)
    # Since we have only submitted one request check 
    # the first request on the list
    if exports[0].status == 'Done':
        print("Export succeded")
        download_uri = exports[0].download_uri
        break
    elif exports[0].status == 'Failed':
        print("Export failed")
        break
    # Wait a little bit
    print("Going to sleep for a few seconds")
    time.sleep(10)
        

### Download the package

In [ ]:
import wget
import os

download_filename = 'docker_export.zip'

if download_uri != None:
    print("Downloading from: {0}".format(download_uri))
    wget.download(download_uri, download_filename)

In [ ]:
%%sh
ls -l 

In [ ]:
%%sh
unzip docker_export.zip -d docker_export

In [ ]:
%%sh
ls -l docker_export

### Review the exported package
Your instructor will guide you through the content of the exported package


## Create and deploy docker image

As explained in more detail by the instructor, the exported package includes a pre-configured *DockerFile*. You are free to modify it but in this lab we will use it as it is.
### Create docker image 

In [2]:
%%sh
cd docker_export

sudo docker build . -t aerialclassifier

"docker build" requires exactly 1 argument.
See 'docker build --help'.

Usage:  docker build [OPTIONS] PATH | URL | -

Build an image from a Dockerfile


### Start docker container
We will first test the image locally.

In [5]:
%%sh

sudo docker run -p 127.0.0.1:80:80 -d aerialclassifier


6473dfa2d9f3460c234ae6515fd868bba4cf7a992f5019b373ef226b5b873c75


In [7]:
%%sh
sudo docker ps 

CONTAINER ID        IMAGE               COMMAND                  CREATED              STATUS              PORTS                  NAMES
6473dfa2d9f3        aerialclassifier    "/bin/sh -c 'python …"   About a minute ago   Up About a minute   127.0.0.1:80->80/tcp   musing_archimedes


### Invoke the web service

In [9]:
%%sh

ls -l samples

total 252
-rw-r--r-- 1 demouser demouser 91989 Sep 25 21:30 barren-1.png
-rw-r--r-- 1 demouser demouser 66363 Sep 25 21:30 cultivated-1.png
-rw-r--r-- 1 demouser demouser 92096 Sep 25 21:30 developed-1.png


In [10]:
%%sh

curl -X POST http://127.0.0.1/image -F imageData=@samples/barren-1.png

{
  "created": "2018-10-09T17:31:02.773790", 
  "id": "", 
  "iteration": "", 
  "predictions": [
    {
      "boundingBox": null, 
      "probability": 0.28911370038986206, 
      "tagId": "", 
      "tagName": "Barren"
    }, 
    {
      "boundingBox": null, 
      "probability": 0.7108862996101379, 
      "tagId": "", 
      "tagName": "Cultivated"
    }
  ], 
  "project": ""
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 92583  100   385  100 92198    312  74755  0:00:01  0:00:01 --:--:-- 74775
